In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas(desc='apply')
import dill
import os
import sys

from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

In [7]:
def iter_create_gensim_lsa_model(files, number_of_topics, dictionary, save=False):
    """
    Input  : clean document, number of topics and number of words associated with each topic
    Purpose: create LSA model using gensim
    Output : return LSA model
    """
    lsamodel = LsiModel(corpus=None, num_topics=number_of_topics, id2word=dictionary)
    for file in tqdm(files, desc='feathers'):
        df = pd.read_feather(file, columns=['title'])
        titles = df['title'].str.split()
        doc_term_matrix = [dictionary.doc2bow(text) for text in titles]
        lsamodel.add_documents(tqdm(doc_term_matrix, desc='model'))
    if save:
        with open('transforms/lsamodel.pkd') as file:
            dill.dump(lsamodel, file)
    return lsamodel

def create_gensim_lsa_model(doc_term_matrix, number_of_topics, dictionary, save=False):
    """
    Input  : clean document, number of topics and number of words associated with each topic
    Purpose: create LSA model using gensim
    Output : return LSA model
    """
    lsamodel = LsiModel(corpus=doc_term_matrix, num_topics=number_of_topics, id2word=dictionary)
    if save:
        with open('transforms/lsamodel.pkd', 'wb') as file:
            dill.dump(lsamodel, file)
    return lsamodel

In [8]:
%%time

folder = 'cleaned_cache'
files = [os.path.join(folder, 'data_{}.feather'.format(i)) for i in range(len(os.listdir(folder)))]

with open('transforms/dictionary.pkd', 'rb') as file:
    d = dill.load(file)
d.filter_extremes(keep_n=5000)

lsamodel = iter_create_gensim_lsa_model(files, number_of_topics=50, dictionary=d)

/Users/garychen/anaconda3/envs/TDI/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [12]:
with open('transforms/lsamodel.pkd', 'rb') as file:
    test = dill.load(file)

In [ ]:
topics = []
for i, topic in enumerate(lsamodel.get_topics()):
    df = pd.DataFrame({'keyword': list(dictionary.values()), 'weight': topic, 'num': [i]*len(topic)})
    df.sort_values('weight', ascending=False, inplace=True)
    topics.append(df)
    
topics = pd.concat(topics)
topics.set_index(['num', 'keyword'], inplace=True)
topics.reset_index().to_feather('transforms/topics.feather')